In [35]:
import numpy as np
import pandas as pd
from scipy.stats import f_oneway
import glob
import scipy.stats as stats
import pingouin as pg
from itertools import chain

# Biosignals

In [252]:
# path = 'C:/Users/evach/Documents/MSc/Diss_data/3_set_baseline/Final_all/Human/Biosignals/'
# path = 'C:/Users/evach/Documents/MSc/Diss_data/3_set_baseline/Final_all/Latency/Biosignals/'
path = 'C:/Users/evach/Documents/MSc/Diss_data/3_set_baseline/Final_all/Weighted/Biosignals/'

path_1 = path+'/set_1_combine.csv'
path_2 = path+'/set_2_combine.csv'
path_3 = path+'/set_3_combine.csv'


## Reading files into a list

In [253]:

ls = [pd.read_csv(i, index_col=[0]) for i in [path_1, path_2, path_3]]

hr1 = np.asarray(ls[0]['HR'])
hr2 = np.asarray(ls[1]['HR'])
hr3 = np.asarray(ls[2]['HR'])

eda1 = ls[0]['EDA']
eda2 = ls[1]['EDA']
eda3 = ls[2]['EDA']

t1 = ls[0]['TEMP']
t2 = ls[1]['TEMP']
t3 = ls[2]['TEMP']

In [254]:
f_oneway(t1,t2,t3)
f_oneway(t1,t2,t3)
f_oneway(eda1,eda2,eda3)

F_onewayResult(statistic=3.7413228142450174, pvalue=0.026607712472360167)

## Bartlett test
 testing to determine whether equal variance is true, if true then use classic
 ANOVA, otherwise use Welch's ANOVA

In [25]:
print(stats.bartlett(hr1,hr2,hr3))
print(stats.bartlett(eda1,eda2,eda3))
print(stats.bartlett(t1,t2,t3))

BartlettResult(statistic=32.43631678494651, pvalue=9.047806489464891e-08)
BartlettResult(statistic=242.9602020430196, pvalue=1.7452693641906554e-53)
BartlettResult(statistic=16.53730858864339, pvalue=0.00025643013866722417)


## ANOVA

## NEED TO MAKE SURE change the 'REPEAT' for different testing

In [26]:
len(hr1)

60

## REMEMBER TO CHANGE THE REPEAT & weights/latency !!!!!

latency = 60
human = 40
weighted = 40

In [255]:
def anova(repeats = 60, tst = 'Latency', path = path+'/stats/'):
    hr = list(chain(*[hr1,hr2,hr3]))
    eda = list(chain(*[eda1,eda2,eda3]))
    temp = list(chain(*[t1,t2,t3]))

    if tst == 'Latency':
        level_name = ['0 ms', '150 ms', '300 ms']
    else:
        level_name = ['0 KG', '0.5 KG', '1 KG']


    data = [hr, eda, temp]
    name = ['HR','EDA','TEMP']

    hold = [[] for i in range(0,len(data))]
    result = [[] for i in range(0,len(data))]
    pair_result = [[] for i in range(0,len(data))]

    for i in range( 0, len(data)):
        hold[i] = pd.DataFrame({'score': data[i],
                    'group': np.repeat(level_name, repeats)}) 
        result[i] = pg.welch_anova(dv='score', between='group', data=hold[i])
        pair_result[i] = pg.pairwise_gameshowell(dv='score', between='group', data=hold[i])
        
        # result[i].to_csv(path+name[i]+' welch anova'+'.csv')
        # pair_result[i].to_csv(path +name[i] + ' pair wise' +'.csv')
    return result, pair_result


In [237]:
hr = list(chain(*[hr1,hr2,hr3]))
eda = list(chain(*[eda1,eda2,eda3]))
temp = list(chain(*[t1,t2,t3]))
data = [hr, eda, temp]

In [257]:
r = anova(tst ='Weights')

ValueError: All arrays must be of the same length

In [29]:
r[0]

[  Source  ddof1       ddof2         F     p-unc       np2
 0  group      2  111.983188  7.349406  0.001002  0.084141,
   Source  ddof1      ddof2        F    p-unc       np2
 0  group      2  92.483483  3.70135  0.02842  0.040091,
   Source  ddof1       ddof2         F     p-unc      np2
 0  group      2  111.206525  2.247097  0.110485  0.02322]

In [30]:
r[1][1]

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,0 KG,0.5 KG,0.757066,0.141011,0.616055,0.284060,2.168753,60.440230,0.084872,0.393436
1,0 KG,1 KG,0.757066,0.273154,0.483912,0.290527,1.665636,65.917109,0.226050,0.302165
2,0.5 KG,1 KG,0.141011,0.273154,-0.132143,0.075248,-1.756103,82.476946,0.190889,-0.318577


# NASA

In [137]:
# path = 'C:/Users/evach/Documents/MSc/Diss_data/3_set_baseline/Final_all/Human/NASA/manova friendly.csv'
# path = 'C:/Users/evach/Documents/MSc/Diss_data/3_set_baseline/Final_all/Latency/NASA/manova friendly.csv'
path = 'C:/Users/evach/Documents/MSc/Diss_data/3_set_baseline/Final_all/Weighted/NASA/manova friendly.csv'
ls = pd.read_csv(path, index_col=[0])


In [138]:
full_lst = list(chain(*[ls]))
full_lst

['Sets',
 'Mental demand',
 'Physical demand ',
 'Temporal performance ( how hurried)',
 'Performance',
 'Effort ',
 'Frustration',
 'Motivation',
 'bnb score']

In [139]:
def pack_sets():
    name_lst = [[] for i in range(0,len(ls.columns))]
    #['mental', 'physical','tempo','performance','effort','frustration','motivation','BBT']

    for i in range(1,len(ls.columns)):
        individual_test = [ls.iloc[:,i]]
        name_lst[i] = list(chain(*individual_test))
    return name_lst

In [140]:
name_lst = pack_sets()
len(name_lst)

9

In [142]:
hold = []
result=[]
pair_result =[]
level_name = ['0 ms', '150 ms', '300 ms']
repeats = 40

hold = pd.DataFrame({'score': name_lst[1],
                    'group': np.repeat(level_name, repeats)}) 
result = pg.welch_anova(dv='score', between='group', data=hold)
pair_result = pg.pairwise_gameshowell(dv='score', between='group', data=hold)
        


In [143]:
name = ['ignore',
 'Mental demand',
 'Physical demand ',
 'Temporal performance ( how hurried)',
 'Performance',
 'Effort ',
 'Frustration',
 'Motivation',
 
 'BBT']

In [211]:
path = 'C:/Users/evach/Documents/MSc/Diss_data/3_set_baseline/Final_all/Weighted/NASA/'

def anova_nasa(repeats = 40, tst = 'Latency', path = path+'/stats/'):
    hold = [[] for  i in range(0, len(ls.columns))]
    pair_result= [[] for  i in range(0, len(ls.columns))]
    result= [[] for  i in range(0, len(ls.columns))]

    if tst == 'Latency':
        level_name = ['0 ms', '150 ms', '300 ms']
    else:
        level_name = ['0 KG', '0.5 KG', '1 KG']
    
    for i in range(1, len(ls.columns)):
        hold[i] = pd.DataFrame({'score': name_lst[i],
                    'group': np.repeat(level_name, repeats)}) 
        # result[i] = pg.welch_anova(dv='score', between='group', data=hold[i])
        # pair_result[i] = pg.pairwise_gameshowell(dv='score', between='group', data=hold[i])
        # result[i].to_csv(path+name[i]+' welch anova'+'.csv')
        # pair_result[i].to_csv(path +name[i] + ' pair wise' +'.csv')
    return hold#result, pair_result

In [212]:
r = anova_nasa()

In [173]:
pd.Index(r[1]['group']).get_loc('300 ms')

slice(80, 120, None)

In [213]:
s1 = pd.Index(r[1]['group']).get_loc('0 ms')
s2 = pd.Index(r[1]['group']).get_loc('150 ms')
s3 = pd.Index(r[1]['group']).get_loc('300 ms')

In [217]:
d1 = r[1].iloc[s1]#.drop('group', axis =1 ,inplace = True)
d2 = r[1].iloc[s2]#.drop('group', axis =1 ,inplace = True)
d3 = r[1].iloc[s3]#.drop('group', axis =1 ,inplace = True)
d1.drop('group', axis =1 ,inplace = True)


C:\Users\evach\AppData\Local\Temp\ipykernel_33172\1711736038.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1.drop('group', axis =1 ,inplace = True)


In [218]:
d2.drop('group', axis =1 ,inplace = True)
d3.drop('group', axis =1 ,inplace = True)

C:\Users\evach\AppData\Local\Temp\ipykernel_33172\2750597821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2.drop('group', axis =1 ,inplace = True)
C:\Users\evach\AppData\Local\Temp\ipykernel_33172\2750597821.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3.drop('group', axis =1 ,inplace = True)


In [225]:
data_1 = d1.to_numpy()
data_2 = d2.to_numpy()
data_3 = d3.to_numpy()

data_1 = list(chain(*data_1))
data_2 = list(chain(*data_2))
data_3 = list(chain(*data_3))

In [227]:
data_2

[40.0,
 40.0,
 40.0,
 40.0,
 40.0,
 40.0,
 60.0,
 60.0,
 50.0,
 50.0,
 80.0,
 60.0,
 70.0,
 90.0,
 60.0,
 60.0,
 70.0,
 80.0,
 70.0,
 80.0,
 20.0,
 20.0,
 20.0,
 20.0,
 20.0,
 20.0,
 25.0,
 25.0,
 30.0,
 25.0,
 50.0,
 40.0,
 40.0,
 40.0,
 35.0,
 20.0,
 15.0,
 15.0,
 15.0,
 10.0]

In [235]:
f_1 = [data_1, data_2, data_3]
f_1.corr()

AttributeError: 'list' object has no attribute 'corr'

In [231]:
correlation_matrix = np.corrcoef(data_1, data_3, data_2)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

In [232]:
from scipy import stats
num_sets = 3
def general_trend_stat(sets):
  hold = [ [] for i in range(0,num_sets) ]
  stat_frame = [pd.DataFrame() for k in range(num_sets)]

  # for each data set/ condition, calcualte the stats
  for k in range(0, num_sets):
    x = range(0, len(sets[k]))
    for column in sets[k]:
      slope, intercept, r_value, p_value, std_err = stats.linregress(x, sets[k].loc[:,column])
      hold[k].append((slope,r_value,p_value,std_err))

    stat_frame[k] = pd.DataFrame(hold[k], columns = ['slope', 'r_value', 'p_value', 'std_err'])
  return stat_frame


0.8441891794373101

In [228]:
# print(stats.bartlett(hr1,hr2,hr3))
# print(stats.bartlett(eda1,eda2,eda3))
# print(stats.bartlett(t1,t2,t3))

print(stats.bartlett(list[data_1],list[data_2],list[data_3]))

TypeError: object of type 'types.GenericAlias' has no len()

'Sets',
 'Mental demand',
 'Physical demand ',
 'Temporal performance ( how hurried)',
 'Performance',
 'Effort ',
 'Frustration',
 'Motivation',
 'bnb score'

In [148]:
r

([[],
    Source  ddof1      ddof2         F     p-unc       np2
  0  group      2  76.833987  0.362628  0.697029  0.005297,
    Source  ddof1     ddof2         F     p-unc       np2
  0  group      2  77.06607  1.533965  0.222191  0.023919,
    Source  ddof1     ddof2        F    p-unc       np2
  0  group      2  77.91844  1.77546  0.17619  0.028756,
    Source  ddof1      ddof2       F    p-unc       np2
  0  group      2  77.750297  1.1913  0.30931  0.021036,
    Source  ddof1      ddof2        F     p-unc       np2
  0  group      2  75.332933  0.76626  0.468336  0.010279,
    Source  ddof1      ddof2         F     p-unc       np2
  0  group      2  77.547622  2.334764  0.103605  0.036023,
    Source  ddof1      ddof2         F     p-unc       np2
  0  group      2  71.218644  4.163501  0.019496  0.056697,
    Source  ddof1      ddof2        F     p-unc       np2
  0  group      2  77.781943  3.87926  0.024779  0.058917],
 [[],
          A       B  mean(A)  mean(B)   diff        s